In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, sys

from sklearn.model_selection import train_test_split
sys.path.append('../../Share/')
import Model

def random_downsample_Num(X, y=None, n_keep=None, random_state=42):
    np.random.seed(random_state)
    n_samples = X.shape[0]

    if n_keep is None or n_keep > n_samples:
        raise ValueError(f"n_keep must be between 1 and {n_samples}, got {n_keep}")

    idx_keep = np.random.choice(n_samples, size=n_keep, replace=False)
    X_down = X[idx_keep]

    if y is not None:
        y_down = y[idx_keep]
        return X_down, y_down
    else:
        return X_down

# Baseline training on subjects
def feature_analysis(X_lst, y_lst):
    num_trials = len(X_lst)
    all_acc = []
    for t in range(num_trials):
        X, y = X_lst[t].reshape(-1,16,14,1), y_lst[t]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
        feature_set = 14
        feat_acc = []

        for f in range(feature_set):
            X_train_f = X_train[:,:,f,:]
            X_test_f = X_test[:,:,f,:]
            #model = Model.Original_model(input_shape=X_train.shape[1:], num_class=6)
            model = Model.Original_model_1DCNN(input_size=X_train_f.shape[1:], num_class=6)

            history, _ = Model.Train_model(model, X_train_f, y_train, X_test_f, y_test, set_epoch=50, set_batch_size=128, Model_name=None, set_verbose=False, save_model_set=False)
            #plt.plot(history.history['val_accuracy'])
            #plt.plot(history.history['accuracy'])
            #plt.show()

            print(f"\tTrial {t+1}/{num_trials}/feat{f} | Val Acc: {np.max(history.history['val_accuracy'])}")
            feat_acc.append(np.max(history.history['val_accuracy']))
        all_acc.append(feat_acc)

    return all_acc


def baseline_run(X_lst, y_lst):
    num_trials, all_acc_per_session = len(X_lst), []
    feature_set, num_channels = 14, 16

    for t in range(num_trials):
        X, y = X_lst[t].reshape(-1,num_channels,feature_set,1), y_lst[t]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

        model = Model.Original_model(input_shape=X_train.shape[1:], num_class=6)
        history, _ = Model.Train_model(model, X_train, y_train, X_test, y_test, set_epoch=50, set_batch_size=128, Model_name=None, set_verbose=False, save_model_set=False)

        print(f"\tTrial {t+1}/{num_trials} | Val Acc: {np.max(history.history['val_accuracy'])}")
        all_acc_per_session.append(np.max(history.history['val_accuracy']))

    return all_acc_per_session


def baseline_run_specific_T(X_lst, y_lst, trials):
    all_acc_per_session = []
    feature_set, num_channels = 14, 16

    for t in trials:
        X, y = X_lst[t].reshape(-1,num_channels,feature_set,1), y_lst[t]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

        model = Model.Original_model(input_shape=X_train.shape[1:], num_class=6)
        history, _ = Model.Train_model(model, X_train, y_train, X_test, y_test, set_epoch=50, set_batch_size=128, Model_name=None, set_verbose=False, save_model_set=False)

        print(f"\tTrial {t+1}/ | Val Acc: {np.max(history.history['val_accuracy'])}")
        all_acc_per_session.append(np.max(history.history['val_accuracy']))

    return all_acc_per_session


def baseline_run_specific_T_with_one_feature(X_lst, y_lst, trials, f=2):
    all_acc_per_session = []
    feature_set, num_channels = 14, 16

    for t in trials:
        X, y = X_lst[t].reshape(-1,num_channels,feature_set,1), y_lst[t]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
        X_train, X_test = X_train[:,:,f,:], X_test[:,:,f,:]

        model = Model.Original_model_1DCNN(input_size=X_train.shape[1:], num_class=6)
        history, _ = Model.Train_model(model, X_train, y_train, X_test, y_test, set_epoch=50, set_batch_size=128, Model_name=None, set_verbose=False, save_model_set=False)

        print(f"\tTrial {t+1}/ | Val Acc: {np.max(history.history['val_accuracy'])}")
        all_acc_per_session.append(np.max(history.history['val_accuracy']))

    return all_acc_per_session


def inter_session_run(X_lst, y_lst):
    num_trials = len(X_lst)
    all_acc_K = []

    for t in [1, 4, 7, 9]:
        X_train = [X_lst[i].reshape(-1, 16, 14, 1) for i in range(t)]
        X_train = np.concatenate(X_train, axis=0)
        y_train = [y_lst[i] for i in range(t)]
        y_train = np.concatenate(y_train, axis=0)

        X_test = [X_lst[i].reshape(-1, 16, 14, 1) for i in range(t, len(X_lst))]
        X_test = np.concatenate(X_test, axis=0)

        y_test = [y_lst[i] for i in range(t, len(y_lst))]
        y_test = np.concatenate(y_test, axis=0)
        X_test, y_test = random_downsample_Num(X_test, y_test, n_keep=1000, random_state=42)
        print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

        model = Model.Original_model(input_shape=X_train.shape[1:], num_class=6)
        history, _ = Model.Train_model(model, X_train, y_train, X_test, y_test, set_epoch=50, set_batch_size=128, Model_name=None, set_verbose=False, save_model_set=False)

        print(f"\tTrial {t+1}/{num_trials}/feat{f} | Val Acc: {np.max(history.history['val_accuracy'])}")
        all_acc_K.append(np.max(history.history['val_accuracy']))
    return all_acc_K

### Subject 1 버리기 + use only subject 2 and 3

In [3]:
path = 'C:/Users/hml76/Desktop/Jupyter/Federated Prototype Learning/Dataset/Ours_cleaned/'
fname = os.listdir(path)

subjects = {
    "subject1": {"X": [], "Y": []},
    "subject2": {"X": [], "Y": []},
    "subject3": {"X": [], "Y": []},
}

for f in fname:
    for i in range(10):
        X = np.array(pd.read_csv(path + f'{f}/rep{i}_X.csv'))
        y = np.array(pd.read_csv(path + f'{f}/rep{i}_Y.csv'))

        # shuffle
        #indices = np.random.permutation(len(X))
        #X, y = X[indices], y[indices]

        # convert one-hot to integer labels
        y_int = np.argmax(y, axis=1)
        num_classes = np.max(y_int) + 1

        # example rule to assign data to subject dict
        if "subject1" in f:
            subjects["subject1"]["X"].append(X)
            subjects["subject1"]["Y"].append(y_int)
        elif "subject2" in f:
            subjects["subject2"]["X"].append(X)
            subjects["subject2"]["Y"].append(y_int)
        elif "subject3" in f:
            subjects["subject3"]["X"].append(X)
            subjects["subject3"]["Y"].append(y_int)

1. K1/4/7/10 initialization (저걸로 학습시키기) + Test on rest of the subjects
2.

# Baseline - S2, S3

In [ ]:
X_lst2, y_lst2 = subjects["subject2"]["X"], subjects["subject2"]["Y"]
X_lst3, y_lst3 = subjects["subject3"]["X"], subjects["subject3"]["Y"]

S2_baseline = baseline_run(X_lst2, y_lst2)
S3_baseline = baseline_run(X_lst3, y_lst3)

In [ ]:
S2_baseline_Ts = baseline_run_specific_T(X_lst2, y_lst2, [1,4,7,9])
S3_baseline_Ts = baseline_run_specific_T(X_lst3, y_lst3, [1,4,7,9])

In [17]:
S2_baseline_Ts_f = baseline_run_specific_T_with_one_feature(X_lst2, y_lst2, [1,4,7], f=2)
S3_baseline_Ts_f = baseline_run_specific_T_with_one_feature(X_lst3, y_lst3, [1,4,7], f=2)

Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 48.93%
Maximum validation accuracy : 58.98%
	Trial 2/ | Val Acc: 0.5897821187973022
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 70.46%
Maximum validation accuracy : 78.15%
	Trial 5/ | Val Acc: 0.78152996301651
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 44.96%
Maximum validation accuracy : 50.57%
	Trial 8/ | Val Acc: 0.5057008862495422
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 61.69%
Maximum validation accuracy : 69.04%
	Trial 2/ | Val Acc: 0.6903553009033203
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 37.58%
Maximum validation accuracy : 45.35%
	Trial 5/ | Val Acc: 0.4534534513950348
Start Training (total epochs: 50)...
Finish Training! (Model i

In [19]:
S2_baseline_Ts_f, S3_baseline_Ts_f

([np.float64(0.5897821187973022),
  np.float64(0.78152996301651),
  np.float64(0.5057008862495422)],
 [np.float64(0.6903553009033203),
  np.float64(0.4534534513950348),
  np.float64(0.6592639684677124)])

In [9]:
S2_baseline, S3_baseline

([np.float64(0.38778746128082275),
  np.float64(0.1743050366640091),
  np.float64(0.7727593779563904),
  np.float64(0.473270446062088),
  np.float64(0.7787732481956482),
  np.float64(0.17456021904945374),
  np.float64(0.17270788550376892),
  np.float64(0.18846412003040314),
  np.float64(0.18199867010116577),
  np.float64(0.4073053002357483)],
 [np.float64(0.6608116030693054),
  np.float64(0.175761416554451),
  np.float64(0.18091079592704773),
  np.float64(0.1728624552488327),
  np.float64(0.17657656967639923),
  np.float64(0.1765071451663971),
  np.float64(0.1748933643102646),
  np.float64(0.7290608882904053),
  np.float64(0.17831943929195404),
  np.float64(0.17969734966754913)])

In [13]:
S2_baseline_Ts, S3_baseline_Ts

([np.float64(0.42524418234825134),
  np.float64(0.8022053837776184),
  np.float64(0.18846412003040314),
  np.float64(0.18814609944820404)],
 [np.float64(0.175761416554451),
  np.float64(0.17657656967639923),
  np.float64(0.1700507551431656),
  np.float64(0.17969734966754913)])

# Inter-session

In [8]:
inter_session_K_S2 = inter_session_run(X_lst2, y_lst2)
inter_session_K_S3 = inter_session_run(X_lst3, y_lst3)

(6303, 16, 14, 1) (6303,) (1000, 16, 14, 1) (1000,)
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 46.11%
Maximum validation accuracy : 21.9%
	Trial 2/10/featsubject3_2019-12-11 Human Trial [19] | Val Acc: 0.21899999678134918
(26399, 16, 14, 1) (26399,) (1000, 16, 14, 1) (1000,)
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 61.79%
Maximum validation accuracy : 27.9%
	Trial 5/10/featsubject3_2019-12-11 Human Trial [19] | Val Acc: 0.27900001406669617
(48077, 16, 14, 1) (48077,) (1000, 16, 14, 1) (1000,)
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 50.98%
Maximum validation accuracy : 35.1%
	Trial 8/10/featsubject3_2019-12-11 Human Trial [19] | Val Acc: 0.35100001096725464
(63084, 16, 14, 1) (63084,) (1000, 16, 14, 1) (1000,)
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training 

In [10]:
inter_session_K_S2, inter_session_K_S3

([np.float64(0.21899999678134918),
  np.float64(0.27900001406669617),
  np.float64(0.35100001096725464),
  np.float64(0.19499999284744263)],
 [np.float64(0.17800000309944153),
  np.float64(0.1850000023841858),
  np.float64(0.1860000044107437),
  np.float64(0.2919999957084656)])

In [ ]:
inter_session_K_S2 = inter_session_run(X_lst2, y_lst2)
inter_session_K_S3 = inter_session_run(X_lst3, y_lst3)